In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
!pip install pyarrow

In [3]:
df =  pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet", engine='pyarrow')

In [4]:
len( df.columns)

19

In [5]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [6]:
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).astype('timedelta64[m]')

In [8]:
df[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'duration']].head()

,tpep_pickup_datetime,tpep_dropoff_datetime,duration
0,2023-01-01 00:32:10,2023-01-01 00:40:36,8.0
1,2023-01-01 00:55:08,2023-01-01 01:01:27,6.0
2,2023-01-01 00:25:04,2023-01-01 00:37:49,12.0
3,2023-01-01 00:03:48,2023-01-01 00:13:25,9.0
4,2023-01-01 00:10:29,2023-01-01 00:21:19,10.0


In [9]:
df['duration'].describe()

count    3.066766e+06
mean     1.518255e+01
std      4.259450e+01
min     -3.000000e+01
25%      7.000000e+00
50%      1.100000e+01
75%      1.800000e+01
max      1.002900e+04
Name: duration, dtype: float64

In [10]:
print("Percetage of data after removing outliers")
len(df[(df['duration']>=1) & (df['duration']<=60) ])/len(df)*100

Percetage of data after removing outliers


98.18303711466737

In [11]:
df= df[(df['duration']>=1) & (df['duration']<=60) ]

In [21]:
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)

train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [13]:
X_train.shape

(3011044, 515)

In [14]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
 
rmse = mean_squared_error(y_train, y_pred, squared=False)
print("RMSE",rmse)

RMSE 7.688651659305238


In [15]:
df_val = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet", engine='pyarrow')

In [16]:
df_val['duration'] = (df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']).astype('timedelta64[m]')

In [26]:
df_val= df_val[(df_val['duration']>=1) & (df_val['duration']<=60) ]

In [27]:
train_dicts_val = df_val[categorical].to_dict(orient='records')

X_train_val = dv.transform(train_dicts_val)

In [28]:
X_train_val.shape

(2858178, 515)

In [29]:
y_train_val = df_val[target].values

In [30]:
y_pred_val = lr.predict(X_train_val)
rmse = mean_squared_error(y_train_val, y_pred_val, squared=False)
print("RMSE",rmse)

RMSE 13.528920239092544
